In [ ]:
#!pip install optuna hyperopt

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
import re
import os
import pandas as pd
from datetime import datetime, timedelta
import random
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, RepeatedKFold
from datetime import datetime
import pickle
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
import warnings
warnings.filterwarnings('ignore')

### Import data

In [ ]:
train_df = pd.read_pickle('../asset/train_df.pkl')
test_df = pd.read_pickle('../asset/test_df.pkl')

### Helper

In [ ]:
def convert_all_features(data):
    
    """This function will help us to convert boolean and cateogrical values to numerical values.
    
    Input:
    data -> dataframe (the original dataframe)
    
    Output:
    final_df -> dataframe (after feature conversions including one hot encoding and ordinal encoding)
    """
    
    # selecting features that we will use for the Prophet model
    features = list(data.columns)
    
    # treat train and test dataset in a different way
    if 'sales' in data.columns:
        features.append('sales')
        
    oe = OrdinalEncoder()

    not_transformed = []
    no_need_to_transform = []
    
    
    # based on the features, use different methods for encoders. 
    for col, dtype in data.dtypes.items():
        if col in ['date', 'unique_key', 'date_year', 'date_quarter', 'date_month', 'date_day', 'date_week', 'year_month'] or dtype in ['datetime64[ns]', 'timedelta64[ns]']:
            no_need_to_transform.append(col)
        elif col == 'transferred':
            data.loc[:, col] = data[col].apply(lambda x: 1 if x == True else 0)
        elif col in ['state_sales_cut', 'store_sales_bins', 'store_type_sales',
                     'family_sales_bins', 'onpromo_avg_bins', 'cluster_sales_indicator']:
            data.loc[:, col] = oe.fit_transform(data[col].values.reshape(-1,1))
        else:
            not_transformed.append(col) 
    dummy_df = pd.get_dummies(data.drop(columns = no_need_to_transform))
    return dummy_df

In [ ]:
def time_split(data, train_size):
    
    """This function will help us create train test split in time series.
    
    Input:
    data -> dataframe (the original dataframe)
    train_size -> float (the percentage of train_set in our dataset)
    
    Output:
    train_df -> dataframe (train_set based on the train_size)
    test_df -> dataframe (test_set based on the train_size)"""
    
    total_row = len(data)
    train_idx = int(total_row * train_size)
    
    train = data[:train_idx]
    test = data[train_idx:]
    
    return train, test

In [ ]:
def calculate_total_errors(pred_dict, actual, yhat):
    
    """This function will calculate the erorrs using mean absolute erorrs.
    
    Input:
    pred_dict -> dict (key: unique_key, values: y and yhat)
    actual -> (values: the actual y value)
    yhat -> (values: the predicdted value from the model)
    
    Output:
    error -> int (mean absolute erorr)"""
    

    
    error_total = 0
    # looping through the dictionary and summing the error.
    for key in pred_dict:
        error = mean_absolute_error(pred_dict[key][actual], pred_dict[key][yhat])
        error_total += error
        
    return error_total

In [ ]:
def analyze_results(original_df, analyze_df):
    
    """This function will look into different predictions and their original distribtuion.
    
    Input:
    origianl_df -> dataframe (original dataframe before the training)
    analyze_df -> dataframe (dataframe that has information regarding prediction)
    
    Output:
    summary_df -> series (shows the summary statistics for the given stores and departments)"""
    
    key_lists = list(analyze_df['unique_key'])
    
    for key in key_lists:
        data = original_df[original_df.unique_str_dep_key == key]
        sales_only = data['sales']
        
        print(f"Summary Statistics for {key}:")
        print(sales_only.describe())

In [ ]:
def combine_info(train_dict, test_dict):
    
    """This function will return a combination of y and y hat including dates from the XGBoost model.
       
       Input:
       
       train_dict -> dictionary (contains information regarding date, y and y hat)
       test_dict -> dictionary (contains information regarding date, y and y hat)
       
       Output:
       
       combined_df -> dataframe (contains information from the both datasets)"""
    
    # create a dictionary to save the results
    combined_df = pd.DataFrame()
    
    # looping through
    for key in train_dict:
        # create temporary dataframes for both train and test
        # also create a column to indicate if they come from the training set or test set.
        train = pd.DataFrame(train_dict[key])
        train.loc[:,'ind'] = 'train'
        test = pd.DataFrame(test_dict[key])
        test.loc[:,'ind'] = 'test'
        total_df = pd.concat([train, test])
        
        # combine the dataframes together
        combined_df = combined_df.append(total_df, ignore_index= True)
        
    return combined_df

### Hyper parameter tuning

- filter out zeros

In [ ]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1, 9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform('n_estimators', 100, 600, 50),
        'seed': 42}

In [ ]:
def objective(space):
    
    clf=XGBRegressor(
                    n_estimators =int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']),
                    n_jobs = 16)
    
    evaluation = [( train_X, train_y), ( test_X, test_y)]
    
    clf.fit(train_X, 
            train_y,
            eval_set=evaluation, 
            eval_metric="rmse",
            early_stopping_rounds=10,
            verbose=False)
    

    pred = clf.predict(test_X)
    rmse = np.sqrt(mean_squared_error(test_y, pred))
    
    print ("SCORE:", rmse)
    return {'loss': rmse, 'status': STATUS_OK }

In [ ]:
np.random.seed(42)

# define space
space={'max_depth': hp.quniform("max_depth", 2, 20, 1),
        'gamma': hp.uniform ('gamma', 1, 7),
        'reg_alpha' : hp.quniform('reg_alpha', 20, 180, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'subsample'        :hp.uniform('subsample', 0.5, 1),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 12, 1),
        'n_estimators': hp.quniform('n_estimators', 100, 600, 30),
        'seed': 42
    }

# change the name 
# causes an issue when saving the parameters, so change the special character / to &
train_df.loc[:, 'family'] = train_df.family.apply(lambda x: str(x).replace('/', '&'))

# create dictionaries to save the results
xgb_params_dict = {}

# create unique keys based on the store number and departments
train_df.loc[:, 'unique_str_dep_key'] = train_df.store_nbr.apply(lambda x: str(x)) + '-' + train_df.family.apply(lambda x: str(x))

# get the unique list of keys
unique_key_list = list(train_df.unique_str_dep_key.unique())

# filter stores that have 0 sales
total_sales_df = train_df.groupby('unique_str_dep_key').sum()[['sales']]
zero_sales = total_sales_df[total_sales_df.sales == 0]

# find stores that have zero sales total
zero_list = list(zero_sales.index)

# we want to filter out data based on the unique keys
for key in unique_key_list:
    
    if key not in zero_list:
        # filter out data
        data = train_df[train_df.unique_str_dep_key == key]
        data.reset_index().drop(columns = ['index', 'date', 'store_nbr', 'family'])

        # perform feature engineering -> one hot coding
        new_data = convert_all_features(data)

        # train test split (70:30)
        train, test = time_split(new_data, 0.7)
        train.drop_duplicates(inplace = True)
        test.drop_duplicates(inplace = True)

        # assign predictors and target variables
        train_X, train_y = train.drop(columns = ['sales']), train.sales.values
        test_X, test_y = test.drop(columns = ['sales']), test.sales.values
        
        
        trials = Trials()
        best_hyperparams = fmin(fn = objective,
                                space = space,
                                algo = tpe.suggest,
                                max_evals = 100,
                                trials = trials)
        
        print(f"{key} found best params.")
        # save the best results
        xgb_params_dict[key] = best_hyperparams
        
        # save the best results as pickle objects
        with open(f'../asset/best_params_optuna/best_params_{key}.pkl', 'wb') as f:
            pickle.dump(xgb_params_dict[key], f)
        
    else:
        # meaning these store + dep combinations have 0 sales
        xgb_params_dict[key] = 'zero_coef'
        
    

- change the data type

In [ ]:
for u_key in xgb_params_dict:
    if isinstance(xgb_params_dict[u_key], str):
        pass
    else:
        for key, val in xgb_params_dict[u_key].items():
            if key in ['max_depth', 'min_child_weight', 'n_estimators']:
                xgb_params_dict[u_key][key] = int(val)

### Use hyper parameters in the model

actual and y hat are the issues

In [ ]:
# create dictionaries to save the results
xgb_pred_dict_train = {}
xgb_pred_dict_test = {}

# loop through the unique key
for u_key in xgb_params_dict:
    
    # filter dataframe so that we can only get the unique df
    data = train_df[train_df.unique_str_dep_key == u_key]
    total_row = len(data)
    train_idx = int(0.7 * total_row)
    
    data.reset_index().drop(columns = ['index', 'date', 'store_nbr', 'family'])

    # perform feature engineering -> one hot coding
    new_data = convert_all_features(data)

    # train test split (70:30)
    train, test = time_split(new_data, 0.7)
    train.drop_duplicates(inplace = True)
    test.drop_duplicates(inplace = True)
    
    # assign predictors and target variables
    train_X, train_y = train.drop(columns = ['sales']), train.sales.values
    test_X, test_y = test.drop(columns = ['sales']), test.sales.values
    
    # if values == zero_coef, then pass
    if isinstance(xgb_params_dict[u_key], str):
        pass
    else:
        # if values inside the params dict are not zero_coef, then used the best hyper params to train the model
        best_params = xgb_params_dict[u_key]
        xgbr = XGBRegressor(**best_params)
        xgbr.fit(train_X, train_y)
        
        train_y_hat = xgbr.predict(train_X)
        test_y_hat = xgbr.predict(test_X)
        
        # save the results based on traning and validation
        xgb_pred_dict_train[u_key] = {'actual':train_y,
                                      'yhat':train_y_hat}
        
        xgb_pred_dict_test[u_key] = {'actual':test_y,
                                     'yhat':test_y_hat}

Combine the results

Find the total error

In [ ]:
total = 0

for key in xgb_pred_dict_test:
    error = mean_absolute_error(xgb_pred_dict_test[key]['actual'], xgb_pred_dict_test[key]['yhat'])
    total += error
print(total)

In [ ]:
error_dict = {}

for key in xgb_pred_dict_test:
    error_dict[key] = mean_absolute_error(xgb_pred_dict_test[key]['actual'], xgb_pred_dict_test[key]['yhat'])

In [ ]:
xgb_test_error_df = pd.DataFrame(error_dict.items())

In [ ]:
xgb_test_error_df.columns = ['unique_key', 'mae']

Look into individual models

In [ ]:
top10_worst = xgb_test_error_df.sort_values(by = 'mae', ascending = False).head(10)

In [ ]:
top10_best = xgb_test_error_df[xgb_test_error_df.mae != 0].sort_values(by = 'mae').head(10)

In [ ]:
analyze_results(train_df, top10_worst)

In [ ]:
analyze_results(train_df, top10_best)

Looking at the summary stats, best top 10 has the lowest mean and standard deviation, while the top 10 worst has relatively higher means and standard deviations.

### Visualization

In [ ]:
worst_list = list(top10_worst.unique_key)

In [ ]:
best_list = list(top10_best.unique_key)

In [ ]:
max(datelist) + timedelta(days = 1)

In [ ]:
combined_df = pd.DataFrame()
start_date = pd.to_datetime('2013-01-01')

for key in xgb_pred_dict_test:
    # if unique key in either best or worst, let's take a look
    
    if key in worst_list or key in best_list:
        train = pd.DataFrame(xgb_pred_dict_train[key])
        train.loc[:, 'ind'] = 'train'
        train.loc[:, 'unique_key'] = key
        
        test = pd.DataFrame(xgb_pred_dict_test[key])
        test.loc[:, 'ind'] = 'test'
        test.loc[:, 'unique_key'] = key
        
        train_datelist = pd.date_range(start_date, periods=len(xgb_pred_dict_train[key]['actual'])).tolist()
        train.loc[:, 'date'] = train_datelist

        test_start_date = max(datelist) + timedelta(days = 1)
        test_datelist = pd.date_range(test_start_date, periods= len(xgb_pred_dict_test[key]['actual'])).tolist()
        test.loc[:, 'date'] = test_datelist
        
        # combine datasets together
        total = pd.concat([train,test])
        combined_df = combined_df.append(total, ignore_index= True)

In [ ]:
combined_df.loc[:, 'which_list'] = combined_df.apply(lambda row: 'best' if row['unique_key'] in best_list else 'worst', axis = 1)

Best Performance

In [ ]:
plot_name =  '../asset/plots/best_xgb_forecasts.pdf'
pp = PdfPages(plot_name)

for key in best_list:
    
    # filter out data
    data = combined_df[combined_df.unique_key == key]
    plt.figure(figsize = (12, 8))
    plt.title(f"Analyze Predictions for {key}", fontsize = 18)
    
    sns.lineplot(x = 'date',
                 y = 'actual',
                 color = 'grey',
                 alpha = 0.7,
                 data = data,
                 label = 'actual')
    
    sns.lineplot(x = 'date',
                 y = 'yhat',
                 color = 'blue',
                 alpha = 0.7,
                 data = data[data.ind == 'train'],
                 label = 'train')
    
    sns.lineplot(x = 'date',
                 y = 'yhat',
                 color = 'red',
                 alpha = 0.7,
                 data = data[data.ind == 'test'],
                 label = 'test')
    sns.despine()
    plt.legend()
    pp.savefig(plt.gcf())           


pp.close()

Worst Performance

In [ ]:
plot_name =  '../asset/plots/worst_xgb_forecasts.pdf'
pp = PdfPages(plot_name)

for key in worst_list:
    
    # filter out data
    data = combined_df[combined_df.unique_key == key]
    plt.figure(figsize = (12, 8))
    plt.title(f"Analyze Predictions for {key}", fontsize = 18)
    
    sns.lineplot(x = 'date',
                 y = 'actual',
                 color = 'grey',
                 alpha = 0.9,
                 data = data,
                 label = 'actual')
    
    sns.lineplot(x = 'date',
                 y = 'yhat',
                 color = 'blue',
                 alpha = 0.5,
                 data = data[data.ind == 'train'],
                 label = 'train')
    
    sns.lineplot(x = 'date',
                 y = 'yhat',
                 color = 'red',
                 alpha = 0.5,
                 data = data[data.ind == 'test'],
                 label = 'test')
    sns.despine()
    plt.legend()
    pp.savefig(plt.gcf())           


pp.close()

Although the errors seem to be huge in the worst prediction list, our test prediction did not seem to be that bad. The reason why we have good results on the best_list is that most of the times the sales in those departments were relatively low, but they seem to have seasonal operations. 